This is the first model used to generate synthetic dataset, even before using GAN 

In probability theory and statistics, a copula is a multivariate cumulative distribution function for which the marginal probability distribution of each variable is uniform on the interval [0, 1]. Copulas are used to describe/model the dependence (inter-correlation) between random variables.[1]
https://en.wikipedia.org/wiki/Copula_%28probability_theory%29

In mathematical terms, a copula is a distribution over the unit cube [0,1]d which is constructed from a multivariate normal distribution over Rd by using the probability integral transform. Intuitively, a copula is a mathematical function that allows us to describe the joint distribution of multiple random variables by analyzing the dependencies between their marginal distributions.

In [1]:
from sdv.demo import load_tabular_demo

In [2]:
data = load_tabular_demo('student_placements')

In [3]:
data.head()

,student_id,gender,second_perc,high_perc,high_spec,degree_perc,degree_type,work_experience,experience_years,employability_perc,mba_spec,mba_perc,salary,placed,start_date,end_date,duration
0,17264,M,67.00,91.00,Commerce,58.00,Sci&Tech,False,0,55.0,Mkt&HR,58.80,27000.0,True,2020-07-23,2020-10-12,3.0
1,17265,M,79.33,78.33,Science,77.48,Sci&Tech,True,1,86.5,Mkt&Fin,66.28,20000.0,True,2020-01-11,2020-04-09,3.0
2,17266,M,65.00,68.00,Arts,64.00,Comm&Mgmt,False,0,75.0,Mkt&Fin,57.80,25000.0,True,2020-01-26,2020-07-13,6.0
3,17267,M,56.00,52.00,Science,52.00,Sci&Tech,False,0,66.0,Mkt&HR,59.43,NaN,False,NaT,NaT,NaN
4,17268,M,85.80,73.60,Commerce,73.30,Comm&Mgmt,False,0,96.8,Mkt&Fin,55.50,42500.0,True,2020-07-04,2020-09-27,3.0


You will notice that there is data with the following characteristics:

    There are float, integer, boolean, categorical and datetime values.

    There are some variables that have missing data. In particular, all the data related to the placement details is missing in the rows where the student was not placed.


Let us use the GaussianCopula to learn this data and then sample synthetic data about new students to see how well the model captures the characteristics indicated above. In order to do this you will need to:

    Import the sdv.tabular.GaussianCopula class and create an instance of it.

    Call its fit method passing our table.

    Call its sample method indicating the number of synthetic rows that you want to generate.


In [4]:
from sdv.tabular import GaussianCopula
import time

model = GaussianCopula()
model.fit(data)

start_time = time.time()
print("--- %s seconds ---" % (time.time() - start_time))

/home/enas/CopulaGAN/lib/python3.7/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)
/home/enas/CopulaGAN/lib/python3.7/site-packages/scipy/stats/_continuous_distns.py:4965: RuntimeWarning: divide by zero encountered in true_divide
  return c**2 / (c**2 - n**2)
/home/enas/CopulaGAN/lib/python3.7/site-packages/scipy/stats/_distn_infrastructure.py:2429: RuntimeWarning: invalid value encountered in double_scalars
  Lhat = muhat - Shat*mu
/home/enas/CopulaGAN/lib/python3.7/site-packages/scipy/stats/_continuous_distns.py:621: RuntimeWarning: invalid value encountered in sqrt
  sk = 2*(b-a)*np.sqrt(a + b + 1) / (a + b + 2) / np.sqrt(a*b)
/home/enas/CopulaGAN/lib/python3.7/site-packages/scipy/optimize/minpack.py:175: RuntimeWarn

--- 4.696846008300781e-05 seconds ---


Notice that the model fitting process took care of transforming the different fields using the appropriate Reversible Data Transforms to ensure that the data has a format that the GaussianMultivariate model can handle

# Generate synthetic data from the model

Once the modeling has finished you are ready to generate new synthetic data by calling the sample method from your model passing the number of rows that we want to generate.

In [5]:
new_data = model.sample(200)

This will return a table identical to the one which the model was fitted on, but filled with new data which resembles the original one.

In [6]:
new_data.head()

,student_id,gender,second_perc,high_perc,high_spec,degree_perc,degree_type,work_experience,experience_years,employability_perc,mba_spec,mba_perc,salary,placed,start_date,end_date,duration
0,17374,M,63.53,63.65,Science,64.70,Sci&Tech,False,2,62.48,Mkt&HR,63.54,27600.0,True,2020-02-13,2020-09-19,3.0
1,17385,F,75.98,62.03,Science,68.11,Comm&Mgmt,False,1,85.58,Mkt&HR,77.89,31000.0,True,2020-01-14,2020-07-22,12.0
2,17472,F,83.01,69.77,Science,78.12,Sci&Tech,False,2,85.77,Mkt&Fin,66.46,32500.0,True,2020-01-30,2020-06-15,3.0
3,17444,M,66.83,66.14,Commerce,58.88,Comm&Mgmt,False,0,51.69,Mkt&HR,66.87,NaN,False,NaT,NaT,NaN
4,17431,M,59.94,58.36,Science,64.99,Sci&Tech,False,1,59.20,Mkt&Fin,58.33,NaN,False,NaT,NaT,NaN


ou can control the number of rows by specifying the number of samples in the model.sample(<num_rows>). To test, try model.sample(10000). Note that the original table only had ~200 rows.

# Save and Load the model

In many scenarios it will be convenient to generate synthetic versions of your data directly in systems that do not have access to the original data source. For example, if you may want to generate testing data on the fly inside a testing environment that does not have access to your production database. In these scenarios, fitting the model with real data every time that you need to generate new data is feasible, so you will need to fit a model in your production environment, save the fitted model into a file, send this file to the testing environment and then load it there to be able to sample from it.

Let’s see how this process works.

# Save and share the model

Once you have fitted the model, all you need to do is call its save method passing the name of the file in which you want to save the model. Note that the extension of the filename is not relevant, but we will be using the .pkl extension to highlight that the serialization protocol used is pickle.

In [7]:
model.save('my_model1.pkl')

This will have created a file called my_model1.pkl in the same directory in which you are running SDV.

# Important

If you inspect the generated file you will notice that its size is much smaller than the size of the data that you used to generate it. This is because the serialized model contains no information about the original data, other than the parameters it needs to generate synthetic versions of it. This means that you can safely share this my_model.pkl file without the risk of disclosing any of your real data!


# Load the model and generate new data

The file you just generated can be sent over to the system where the synthetic data will be generated. Once it is there, you can load it using the GaussianCopula.load method, and then you are ready to sample new data from the loaded instance:

In [11]:
loaded = GaussianCopula.load('my_model1.pkl')

In [12]:
new_data = loaded.sample(200)

Notice that the system where the model is loaded needs to also have sdv installed, otherwise it will not be able to load the model and use it.

# Specifying the Primary Key of the table

One of the first things that you may have noticed when looking at the demo data is that there is a student_id column which acts as the primary key of the table, and which is supposed to have unique values. Indeed, if we look at the number of times that each value appears, we see that all of them appear at most once:

In [13]:
data.student_id.value_counts().max()

1

However, if we look at the synthetic data that we generated, we observe that there are some values that appear more than once:

In [ ]:
new_data[new_data.student_id == new_data.student_id.value_counts().index[0]]

This happens because the model was not notified at any point about the fact that the student_id had to be unique, so when it generates new data it will provoke collisions sooner or later. In order to solve this, we can pass the argument primary_key to our model when we create it, indicating the name of the column that is the index of the table.

In [ ]:
model = GaussianCopula(
   ....:     primary_key='student_id'
   ....: )
   ....: 

In [ ]:
model.fit(data)
print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
new_data = model.sample(200)

In [ ]:
new_data.head()

As a result, the model will learn that this column must be unique and generate a unique sequence of values for the column:

In [ ]:
new_data.student_id.value_counts().max()

# Anonymizing Personally Identifiable Information (PII)

There will be many cases where the data will contain Personally Identifiable Information which we cannot disclose. In these cases, we will want our Tabular Models to replace the information within these fields with fake, simulated data that looks similar to the real one but does not contain any of the original values.

Let’s load a new dataset that contains a PII field, the student_placements_pii demo, and try to generate synthetic versions of it that do not contain any of the PII fields.

# Note

The student_placements_pii dataset is a modified version of the student_placements dataset with one new field, address, which contains PII information about the students. Notice that this additional address field has been simulated and does not correspond to data from the real users.


In [ ]:
data_pii = load_tabular_demo('student_placements_pii')

In [ ]:
data_pii.head()

In [ ]:
model = GaussianCopula(
   ....:     primary_key='student_id',
   ....: )
   ....: 

In [ ]:
model.fit(data_pii)
print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
new_data_pii = model.sample(200)

In [ ]:
new_data_pii.head()

In [ ]:
new_data_pii.address.isin(data_pii.address).sum()

In order to solve this, we can pass an additional argument anonymize_fields to our model when we create the instance. This anonymize_fields argument will need to be a dictionary that contains:

    The name of the field that we want to anonymize.

    The category of the field that we want to use when we generate fake values for it.

The list complete list of possible categories can be seen in the Faker Providers page, and it contains a huge list of concepts such as:

    name

    address

    country

    city

    ssn

    credit_card_number

    credit_card_expire

    credit_card_security_code

    email

    telephone

    …

In this case, since the field is an e-mail address, we will pass a dictionary indicating the category address

In [ ]:
model = GaussianCopula(
   ....:     primary_key='student_id',
   ....:     anonymize_fields={
   ....:         'address': 'address'
   ....:     }
   ....: )
   ....: 

In [ ]:
model.fit(data_pii)
print("--- %s seconds ---" % (time.time() - start_time))

As a result, we can see how the real address values have been replaced by other fake addresses:

In [ ]:
new_data_pii = model.sample(200)

In [ ]:
new_data_pii.head()

Which means that none of the original addresses can be found in the sampled data:

In [ ]:
data_pii.address.isin(new_data_pii.address).sum()

# Advanced Usage

Now that we have discovered the basics, let’s go over a few more advanced usage examples and see the different arguments that we can pass to our GaussianCopula Model in order to customize it to our needs.

# Advanced Usage

Now that we have discovered the basics, let’s go over a few more advanced usage examples and see the different arguments that we can pass to our GaussianCopula Model in order to customize it to our needs.
How to set transforms to use?

One thing that you may have noticed when executing the previous steps is that the fitting process took much longer on the student_placements_pii dataset than it took on the previous version that did not contain the student address. This happens because the address field is interpreted as a categorical variable, which the GaussianCopula one-hot encoded generating 215 new columns that it had to learn afterwards.

This transformation, which in this case was very inefficient, happens because the Tabular Models apply Reversible Data Transforms under the hood to transform all the non-numerical variables, which the underlying models cannot handle, into numerical representations which they can properly work with. In the case of the GaussianCopula, the default transformation is a One-Hot encoding, which can work very well with variables that have a small number of different values, but which is very inefficient in cases where there is a large number of values.

For this reason, the Tabular Models have an additional argument called field_transformers that let you select which transformer to apply to each column. This field_transformers argument must be passed as a dict which contains the name of the fields for which we want to use a transformer different than the default, and the name of the transformer that we want to use.

Possible transformer names are:

    integer: Uses a NumericalTransformer of dtype int.

    float: Uses a NumericalTransformer of dtype float.

    categorical: Uses a CategoricalTransformer without gaussian noise.

    categorical_fuzzy: Uses a CategoricalTransformer adding gaussian noise.

    one_hot_encoding: Uses a OneHotEncodingTransformer.

    label_encoding: Uses a LabelEncodingTransformer.

    boolean: Uses a BooleanTransformer.

    datetime: Uses a DatetimeTransformer.

NOTE: For additional details about each one of the transformers, please visit RDT (https://github.com/sdv-dev/RDT)

Let’s now try to improve the previous fitting process by changing the transformer that we use for the address field to something other than the default. As an example, we will use the label_encoding transformer, which instead of generating one column for each possible value, it just replaces each value with a unique integer value.

In [ ]:
model = GaussianCopula(
   ....:     primary_key='student_id',
   ....:     anonymize_fields={
   ....:         'address': 'address'
   ....:     },
   ....:     field_transformers={
   ....:         'address': 'label_encoding'
   ....:     }
   ....: )
   ....: 


In [ ]:
model.fit(data_pii)
print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
new_data_pii = model.sample(200)

In [ ]:
new_data_pii.head()

# Setting Bounds and Specifying Rounding for Numerical Columns

By default, the model will learn the upper and lower bounds of the input data, and use that for sampling. This means that all sampled data will be between the maximum and minimum values found in the original dataset for each numeric column. This option can be overwritten using the min_value and max_value model arguments. These values can either be set to a numeric value, set to 'auto' which is the default setting, or set to None which will mean the column is boundless.

The model will also learn the number of decimal places to round to by default. This option can be overwritten using the rounding parameter. The value can be an int specifying how many decimal places to round to, 'auto' which is the default setting, or None which means the data will not be rounded.

Since we may want to sample values outside of the ranges in the original data, let’s pass the min_value and max_value arguments as None to the model. To keep the number of decimals consistent across columns, we can set rounding to be 2.

In [ ]:
model = GaussianCopula(
   ....:     primary_key='student_id',
   ....:     min_value=None,
   ....:     max_value=None,
   ....:     rounding=2
   ....: )
   ....: 

In [ ]:
model.fit(data)
print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
unbounded_data = model.sample(10)

In [ ]:
unbounded_data

As you may notice, the sampled data may have values outside the range of the original data.

# Exploring the Probability Distributions

During the previous steps, every time we fitted the GaussianCopula it performed the following operations:

    Learn the format and data types of the passed data

    Transform the non-numerical and null data using Reversible Data Transforms to obtain a fully numerical representation of the data from which we can learn the probability distributions.

    Learn the probability distribution of each column from the table

    Transform the values of each numerical column by converting them to their marginal distribution CDF values and then applying an inverse CDF transformation of a standard normal on them.

    Learn the correlations of the newly generated random variables.

After this, when we used the model to generate new data for our table using the sample method, it did:

    Sample from a Multivariate Standard Normal distribution with the learned correlations.

    Revert the sampled values by computing their standard normal CDF and then applying the inverse CDF of their marginal distributions.

    Revert the RDT transformations to go back to the original data format.

As you can see, during these steps the Marginal Probability Distributions have a very important role, since the GaussianCopula had to learn and reproduce the individual distributions of each column in our table. We can explore the distributions which the GaussianCopula used to model each column using its get_distributions method:

In [ ]:
model = GaussianCopula(
   ....:     primary_key='student_id',
   ....:     min_value=None,
   ....:     max_value=None
   ....: )
   ....: 

In [ ]:
model.fit(data)
print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
distributions = model.get_distributions()

In [ ]:
distributions

# Note

In this list we will see multiple distributions for each one of the columns that we have in our data. This is because the RDT transformations used to encode the data numerically often use more than one column to represent each one of the input variables.


Let’s explore the individual distribution of one of the columns in our data to better understand how the GaussianCopula processed them and see if we can improve the results by manually specifying a different distribution. For example, let’s explore the experience_years column by looking at the frequency of its values within the original data:

In [ ]:
data.experience_years.value_counts()

In [ ]:
data.experience_years.hist();

By observing the data we can see that the behavior of the values in this column is very similar to a Gamma or even some types of Beta distribution, where the majority of the values are 0 and the frequency decreases as the values increase.

Was the GaussianCopula able to capture this distribution on its own?

In [ ]:
distributions['experience_years']

It seems that it was not, as it rather thought that the behavior was closer to a Gaussian distribution. And, as a result, we can see how the generated values now contain negative values which are invalid for this column:

In [ ]:
new_data.experience_years.value_counts()

In [ ]:
new_data.experience_years.hist();

Let’s see how we can improve this situation by passing the GaussianCopula the exact distribution that we want it to use for this column

# Setting distributions for individual variables

The GaussianCopula class offers the possibility to indicate which distribution to use for each one of the columns in the table, in order to solve situations like the one that we just described. In order to do this, we need to pass a field_distributions argument with dict that indicates the distribution that we want to use for each column.

Possible values for the distribution argument are:

    univariate: Let copulas select the optimal univariate distribution. This may result in non-parametric models being used.

    parametric: Let copulas select the optimal univariate distribution, but restrict the selection to parametric distributions only.

    bounded: Let copulas select the optimal univariate distribution, but restrict the selection to bounded distributions only. This may result in non-parametric models being used.

    semi_bounded: Let copulas select the optimal univariate distribution, but restrict the selection to semi-bounded distributions only. This may result in non-parametric models being used.

    parametric_bounded: Let copulas select the optimal univariate distribution, but restrict the selection to parametric and bounded distributions only.

    parametric_semi_bounded: Let copulas select the optimal univariate distribution, but restrict the selection to parametric and semi-bounded distributions only.

    gaussian: Use a Gaussian distribution.

    gamma: Use a Gamma distribution.

    beta: Use a Beta distribution.

    student_t: Use a Student T distribution.

    gaussian_kde: Use a GaussianKDE distribution. This model is non-parametric, so using this will make get_parameters unusable.

    truncated_gaussian: Use a Truncated Gaussian distribution.

Let’s see what happens if we make the GaussianCopula use the gamma distribution for our column.

In [ ]:
from sdv.tabular import GaussianCopula

In [ ]:
model = GaussianCopula(
   ....:     primary_key='student_id',
   ....:     field_distributions={
   ....:         'experience_years': 'gamma'
   ....:     },
   ....:     min_value=None,
   ....:     max_value=None
   ....: )
   ....: 

In [ ]:
model.fit(data)
print("--- %s seconds ---" % (time.time() - start_time))

After this, we can see how the GaussianCopula used the indicated distribution for the experience_years column

In [ ]:
model.get_distributions()['experience_years']

And, as a result, we can see how the generated data now have a behavior which is closer to the original data and always stays within the valid values range.

In [ ]:
new_data = model.sample(len(data))

In [ ]:
new_data.experience_years.value_counts()

In [ ]:
new_data.experience_years.hist();

# Note

Even though there are situations like the one shown above where manually choosing a distribution seems to give better results, in most cases the GaussianCopula will be able to find the optimal distribution on its own, making this manual search of the marginal distributions necessary on very little occasions.


# Conditional Sampling

As the name implies, conditional sampling allows us to sample from a conditional distribution using the GaussianCopula model, which means we can generate only values that satisfy certain conditions. These conditional values can be passed to the conditions parameter in the sample method either as a dataframe or a dictionary.

In case a dictionary is passed, the model will generate as many rows as requested, all of which will satisfy the specified conditions, such as gender = M.

In [ ]:
conditions = {
   ....:     'gender': 'M'
   ....: }
   ....:

In [ ]:
 model.sample(5, conditions=conditions)

It’s also possible to condition on multiple columns, such as gender = M, 'experience_years': 0.

In [ ]:
conditions = {
   ....:     'gender': 'M',
   ....:     'experience_years': 0
   ....: }
   ....: 

In [ ]:
 model.sample(5, conditions=conditions)

The conditions can also be passed as a dataframe. In that case, the model will generate one sample for each row of the dataframe, sorted in the same order. Since the model already knows how many samples to generate, passing it as a parameter is unnecessary. For example, if we want to generate three samples where gender = M and three samples with gender = F, we can do the following:

In [ ]:
import pandas as pd

In [ ]:
conditions = pd.DataFrame({
   ....:     'gender': ['M', 'M', 'M', 'F', 'F', 'F'],
   ....: })
   ....: 